In [ ]:
from Supporting_functions import *
import time

# Predictions for a folder

In [ ]:
# Define paths

modelName = 'nuclei2nuclei'
modelId = 'deblur_net_22_7200.pkl'

# Models
srcpath = r'/home/hmorales/WorkSpace/DataIsoReconstructions/'+modelName+'/'
model_path = srcpath+'checkpoint/saved_models/deblur/'+modelId

# Image to test
img_src_path = '/home/hmorales/WorkSpace/DataIsoReconstructions/SourceData/Isotropic/Nuclei/'
scale=0.23160005

#output dir
outdir = '/home/hmorales/WorkSpace/DataIsoReconstructions/SourceData/Isotropic/Nuclei/'

# CUDA device
device1 = torch.device('cuda:0')
device2 = torch.device('cuda:0')
batch_size = 16

# Image Normalization
min_v = 0
max_v = 65535
norm_percentiles = (20.0, 99.9995)  #99.9995 For Nuclei, 99.999 For Membranes


# Create output folder
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [ ]:
# Prepare networks
deblur_net_A = Self_net_architecture.define_G(input_nc=1, output_nc=1, ngf=64, netG='deblur_net', device=device1,use_dropout=False,norm='instance')
deblur_net_B = Self_net_architecture.define_G(input_nc=1, output_nc=1, ngf=64, netG='deblur_net', device=device1,use_dropout=False,norm='instance')

#deblur_net_A  = Self_net_architecture.define_G(input_nc=1, output_nc=1, ngf=64, netG='deblur_net', device=device1, use_dropout=False, norm='instance')
#deblur_net_B  = Self_net_architecture.define_G(input_nc=1, output_nc=1, ngf=64, netG='deblur_net', device=device1, use_dropout=False, norm='instance')


 # Load Model
deblur_net_A.load_state_dict(torch.load(model_path))
deblur_net_B.load_state_dict(torch.load(model_path))
    

# Get all tif images in the folder
image_names = [f for f in os.listdir(img_src_path) if f.endswith('.tif')]

# Calculate for each model
for i, image_name in enumerate(image_names):

    start_time = time.time()  # Record the start time   
    
    # Open image
    img_path = os.path.join(img_src_path, image_name)
    img  =  tifffile.imread(img_path).astype(np.float32)

    # Normalize 
    img = image_preprocessing(img, norm_percentiles, min_v, max_v)
    
    # Predict image
    fusion_stack = upsample_block(img,scale,1,deblur_net_A,deblur_net_B,min_v,max_v, device1, device2,batch_size)
    fusion_stack = image_preprocessing(fusion_stack, norm_percentiles, min_v, max_v)
    
    
    # Save image
    img_out = os.path.join(outdir, image_name)
    tifffile.imwrite(img_out,fusion_stack)
    
    elapsed_time = time.time() - start_time
    print(f"Elapsed Time: {elapsed_time:.4f} seconds, image {image_name}")   
    
